In [1]:
# 실행 환경 확인

import pandas as pd
import numpy as np
import sklearn
import scipy
import statsmodels
import mlxtend
import sys

print(sys.version)
for i in [pd, np, sklearn, scipy, mlxtend, statsmodels]:
    print(i.__name__, i.__version__)

3.7.4 (tags/v3.7.4:e09359112e, Jul  8 2019, 20:34:20) [MSC v.1916 64 bit (AMD64)]
pandas 0.25.1
numpy 1.18.5
sklearn 0.21.3
scipy 1.5.2
mlxtend 0.15.0.0
statsmodels 0.11.1


# 문제 6

(kaggle 형 문제) prob4 를 바탕으로 아래 데이터셋을 만든다.

. prob6_train: DateHour 변수 기준으로 2021년 8월 14일 전(8월 14일 미포함) 데이터 (행의 수: 5256개)

. prob6_test: DateHour 변수 기준으로 2021년 8월 14일 이후(8월 14일 포함) 데이터 (행의 수: 744 개)

일 때, prob6_train으로 target을 예측하는 모델을 만들어, 

prob6_test의 target에 대한 MAE를 최소화하는 모델을 만든다. 

prob6_test의 예측 결과를 아래와 같은 형식으로 출력한다. 파일명은 answer6.csv 이다.

|DateHour|TotalHour|
|--------|---------|
|2021-08-14 00:00:00|102.607580|
|2021-08-14 01:00:00|94.078890|
....

In [2]:
df_elec = pd.read_csv('elec.csv', parse_dates=['Date', 'DateHour'])
df_info = pd.read_csv('info.csv')
df_info['Date'] = pd.to_datetime(df_info['Date'])

In [3]:
df_elec1 = df_elec.pivot(
    index='DateHour', 
    columns='Minute', 
    values='Value').reset_index() # index에 위치한 DateHour를 컬럼에 위치시킵니다
df_elec1

Minute,DateHour,15분,30분,45분,60분
0,2021-01-01 00:00:00,62,61,61,61
1,2021-01-01 01:00:00,96,93,116,113
2,2021-01-01 02:00:00,106,96,106,107
3,2021-01-01 03:00:00,92,110,110,109
4,2021-01-01 04:00:00,108,105,106,108
...,...,...,...,...,...
6163,2021-09-14 19:00:00,152,151,171,139
6164,2021-09-14 20:00:00,124,130,128,130
6165,2021-09-14 21:00:00,134,130,125,124
6166,2021-09-14 22:00:00,100,109,120,114


In [4]:
holi = pd.to_datetime(["2021-01-01", "2021-02-11", "2021-02-12", "2021-03-01", "2021-05-05", "2021-05-19", "2021-08-16"]).date

# pd.Series.dt accessor를 통해 파생 변수들을 만듭니다.
df_elec1 = df_elec1.assign(
    DayName = df_elec1['DateHour'].dt.weekday, 
    Hour = df_elec1['DateHour'].dt.hour,
    AM = (df_elec1['DateHour'].dt.hour >= 12).astype('int'),
    Weekend_yn = df_elec1['DateHour'].dt.weekday.isin([5, 6]).astype('int'),
    Holiday_yn = df_elec1['DateHour'].dt.date.isin(holi).astype('int'), # df_elec1['DateHour'].dt.date는 pd.Series 입니다.
    Avg = df_elec1.mean(axis=1),
    TotalHour = df_elec1.sum(axis=1),
)

In [5]:
df_info1 = df_info.fillna(0)

In [6]:
df_basetable1 = df_elec1.merge(
    df_info1, 
    left_on='DateHour', 
    right_on='Date', how='inner'
).drop(columns='Date') # DateHour가 컬럼에 있으므로 DateHour, Date 모두 존재하여, Date는 삭제했습니다.
df_basetable1

,DateHour,15분,30분,45분,60분,DayName,Hour,AM,Weekend_yn,Holiday_yn,Avg,TotalHour,생산량,기온,풍속,습도,강수량,전기요금(계절),공장인원,인건비
0,2021-01-01 00:00:00,62,61,61,61,4,0,0,0,1,61.25,245,0,-3.2,2.4,71,0.0,109.8,0.000000,1.5
1,2021-01-01 01:00:00,96,93,116,113,4,1,0,0,1,104.50,418,0,-4.5,1.5,77,0.0,109.8,0.000000,1.5
2,2021-01-01 02:00:00,106,96,106,107,4,2,0,0,1,103.75,415,0,-3.9,2.6,58,0.0,109.8,0.000000,1.5
3,2021-01-01 03:00:00,92,110,110,109,4,3,0,0,1,105.25,421,0,-4.1,2.6,56,0.0,109.8,0.000000,1.5
4,2021-01-01 04:00:00,108,105,106,108,4,4,0,0,1,106.75,427,0,-4.6,2.6,60,0.0,109.8,0.000000,1.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6163,2021-09-14 19:00:00,152,151,171,139,1,19,1,0,0,153.25,613,1497,21.7,3.6,85,9.4,167.2,2.442088,1.5
6164,2021-09-14 20:00:00,124,130,128,130,1,20,1,0,0,128.00,512,45,22.2,4.2,78,9.4,167.2,0.087891,1.5
6165,2021-09-14 21:00:00,134,130,125,124,1,21,1,0,0,128.25,513,149,22.2,4.3,76,9.4,167.2,0.290448,1.5
6166,2021-09-14 22:00:00,100,109,120,114,1,22,1,0,0,110.75,443,66,22.0,2.5,79,9.4,167.2,0.148984,1.5


In [7]:
# index에 DateHour가 위치 하지 않을 경우에는 index 기반한 방법은 구사하기 어렵습니다.
df_prob3 = df_basetable1.sort_values('DateHour')
df_prob3['target'] = df_prob3['TotalHour'].shift(-24)
df_prob4 = pd.concat(
    [df_prob3] + [
        df_prob3['TotalHour'].shift(24 * i).rename('lag_{}'.format(i)) for i in range(1, 7)
    ], axis=1
).dropna()

In [8]:
# 그래도 target만 구해보면 아래와 같은 방법을 생각할 수 있습니다.
df_basetable1.join(
    df_basetable1[['DateHour','TotalHour']].assign(
        DateHour=lambda x: x['DateHour'] - pd.Timedelta(days=1)
    ).set_index('DateHour').rename(columns={'TotalHour': 'target'}),
    on='DateHour', how='inner'
)

,DateHour,15분,30분,45분,60분,DayName,Hour,AM,Weekend_yn,Holiday_yn,...,TotalHour,생산량,기온,풍속,습도,강수량,전기요금(계절),공장인원,인건비,target
0,2021-01-01 00:00:00,62,61,61,61,4,0,0,0,1,...,245,0,-3.2,2.4,71,0.0,109.8,0.000000,1.5,253
1,2021-01-01 01:00:00,96,93,116,113,4,1,0,0,1,...,418,0,-4.5,1.5,77,0.0,109.8,0.000000,1.5,418
2,2021-01-01 02:00:00,106,96,106,107,4,2,0,0,1,...,415,0,-3.9,2.6,58,0.0,109.8,0.000000,1.5,415
3,2021-01-01 03:00:00,92,110,110,109,4,3,0,0,1,...,421,0,-4.1,2.6,56,0.0,109.8,0.000000,1.5,421
4,2021-01-01 04:00:00,108,105,106,108,4,4,0,0,1,...,427,0,-4.6,2.6,60,0.0,109.8,0.000000,1.5,427
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6139,2021-09-13 19:00:00,162,160,148,122,0,19,1,0,0,...,592,2820,22.3,2.1,78,0.0,167.2,4.763514,1.5,613
6140,2021-09-13 20:00:00,113,122,122,126,0,20,1,0,0,...,483,14,22.2,1.3,78,0.0,167.2,0.028986,1.5,512
6141,2021-09-13 21:00:00,122,131,122,115,0,21,1,0,0,...,490,72,21.9,1.9,77,0.0,167.2,0.146939,1.5,513
6142,2021-09-13 22:00:00,97,108,122,113,0,22,1,0,0,...,440,11,21.7,1.4,77,0.0,167.2,0.025000,1.5,443


In [9]:
# 문제에서 제시한 데이터셋을 만듭니다.
df_prob6_train = df_prob4.loc[df_prob4['DateHour'] < '2021-08-14']
df_prob6_test = df_prob4.loc[df_prob4['DateHour'] >= '2021-08-14']

In [10]:
s_kaggle_ans =  df_prob6_test.pop('target')

In [11]:
# Train으로 사용하라고 제시한 prob6_train에서,
# 학습 결과를 검증하기 위해 train / test를 분리해봅니다. 
df_train = df_prob6_train.loc[df_prob6_train['DateHour'] < '2021-07-14']
df_test = df_prob6_train.loc[df_prob6_train['DateHour'] >= '2021-07-14']

In [12]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error

# 모델을 평가하기 위한 함수입니다.
def eval_model(model, X):
    # 입력 변수에 대상변수는 제외합니다.
    X = [i for i in X if i != 'target']
    model.fit(df_train[X], df_train['target'])
    return (
        mean_absolute_error(df_train['target'], model.predict(df_train[X])),
        mean_absolute_error(df_test['target'], model.predict(df_test[X]))
    )

# 모델을 선택하기 위함 함수입니다. 
# 제출 파일을 만들고,
# 자가 채점을 해보기 위한 예측결과를 반환합니다.
def choose_model(model, X):
    # 입력 변수에 대상변수는 제외합니다.
    X = [i for i in X if i != 'target']
    model.fit(df_prob6_train[X], df_prob6_train['target'])
    prd = model.predict(df_prob6_test[X])
    pd.DataFrame({
        'DateHour': df_test['DateHour'],
        'TotalHour': prd
    }).to_csv('answer6.csv', index=None)
    return prd

In [13]:
# Baseline 모델을 만들어 제출용을 만들고 시작합니다.

from sklearn.ensemble import GradientBoostingRegressor

X_gb = "15분, 30분, 45분, 60분, DayName, Hour, AM, Weekend_yn, Holiday_yn, Avg, TotalHour, 생산량, 기온, 풍속, 습도, 강수량, 전기요금(계절), 공장인원, 인건비"
X_gb = [i.strip() for i in  X_gb.split(',')]
ct = ColumnTransformer([
    ('pt', 'passthrough', X_gb)
])
reg_gb = make_pipeline(
    ct, 
    GradientBoostingRegressor(
        loss='huber', # 손실함수: huber
        n_estimators=75, # 트리의 수: 75
        max_depth=7,# 트리의 최대 깊이: 7
        min_samples_leaf=4, # 리프 노드(Leaf Node)의 최소 샘플 수: 4
        random_state=123, # 랜덤 시드: 123
    )
)
eval_model(reg_gb, df_train.columns)

(36.73714649971901, 117.32391481946111)

In [14]:
# 제출 파일을 만듭니다.
prd = choose_model(reg_gb, df_train.columns)

In [15]:
# 자가 채점 결과 입니다.
mean_absolute_error(s_kaggle_ans, prd)

38.86815607791259

In [16]:
# 추가적인 모델을 만들어 봅니다.
# SVR 모델을 만들어 성능 평가를 해봅니다.
ct = ColumnTransformer([
    ('std', StandardScaler(), ['TotalHour'] + ['lag_{}'.format(i) for i in range(1, 7)])
])

X_svr = ['TotalHour'] + ['lag_{}'.format(i) for i in range(1, 7)]
reg_svr = make_pipeline(ct, SVR(C = 10, kernel='rbf', gamma=0.1))

eval_model(reg_svr, df_train.columns)

(61.50534596142382, 148.65000821712516)

In [17]:
from sklearn.decomposition import PCA
# GB보다 성능이 떨어 집니다. 
# DayName을 추가합니다. OneHotEncoder를 사용하여 가변수화 합니다.
# PCA도 추가합니다.
ct = ColumnTransformer([
    ('std', make_pipeline(StandardScaler(), PCA(n_components=0.9)), ['TotalHour'] + ['lag_{}'.format(i) for i in range(1, 7)]),
    ('ohe', OneHotEncoder(drop='first', categories='auto'), ['DayName'])
])

X_svr = ['TotalHour'] + ['lag_{}'.format(i) for i in range(1, 7)]
reg_svr = make_pipeline(ct, SVR(C = 10, kernel='rbf', gamma=0.1))

eval_model(reg_svr, df_train.columns)

(66.81882079820444, 142.46778295852405)

In [18]:
from sklearn.ensemble import VotingRegressor
# GB와 SVR을 VotingRegressor로 앙상블 모델을 구성합니다.
reg_vt = VotingRegressor([
    ('gb', reg_gb),
    ('svr', reg_svr)
])

eval_model(reg_vt, df_train.columns)

(48.96913140811135, 126.49430317665536)

In [19]:
# Baseline 보다 검증 결과 좋지 않지만, 시범삼아 제출파일 만듭니다.
# 채점까지 해봅니다.
prd = choose_model(reg_vt, df_train.columns)
mean_absolute_error(s_kaggle_ans, prd)

29.304664222044433

In [20]:
# RandomForest로 모델을 만들어 봅니다.
from sklearn.ensemble import RandomForestRegressor

X_rf = "15분, 30분, 45분, 60분, DayName, Hour, AM, Weekend_yn, Holiday_yn, Avg, TotalHour, 생산량, 기온, 풍속, 습도, 강수량, 전기요금(계절), 공장인원, 인건비"
X_rf = [i.strip() for i in  X_rf.split(',')]
ct = ColumnTransformer([
    ('pt', 'passthrough', X_rf)
])
reg_rf = make_pipeline(
    ct, 
    RandomForestRegressor(
        n_estimators=200, # 트리의 수: 75
        max_depth=9,# 트리의 최대 깊이: 7
        min_samples_leaf=4, # 리프 노드(Leaf Node)의 최소 샘플 수: 4
        random_state=123, # 랜덤 시드: 123
    )
)
eval_model(reg_rf, df_train.columns)

(54.8373802734469, 106.84973797105972)

In [21]:
# Baseline 보다 좋습니다.
# 제출파일을 갱신하고, 채점도해봅니다.
# 채점결과 좋지는 않습니다.
prd = choose_model(reg_rf, df_train.columns)
mean_absolute_error(s_kaggle_ans, prd)

33.72909173168054

In [22]:
# gb, rf, svr 세 개의 모델을 앙상블해봅니다.
# rf보다 성능이 좋지 않습니다. 버립니다.
reg_vt2 = VotingRegressor([
    ('gb', reg_gb),
    ('rf', reg_rf),
    ('svr', reg_svr)
])

eval_model(reg_vt2, df_train.columns)

(49.6152776441591, 118.1929293916849)